In [2]:
import pandas as pd
from pyecharts.charts import Map,Bar,Line,Grid,Pie,Scatter
from pyecharts import options as opts

# 1.Pandas数据处理
## 1.1读取数据

In [3]:
df=pd.read_csv('./二手房数据.csv',encoding='gb18030')
df.head()

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份
0,朝阳,育慧里一区,1室0厅,西,7,精装,有电梯,52.0,343.0,2001
1,朝阳,大西洋新城A区,2室2厅,南北,10,精装,有电梯,86.0,835.0,1999
2,朝阳,团结湖路,2室1厅,东西,6,精装,无电梯,65.0,430.0,1980
3,朝阳,尚家楼48号院,2室1厅,南北,12,精装,有电梯,75.0,610.0,1998
4,朝阳,望京西园一区,3室2厅,南北,6,精装,无电梯,115.0,710.0,1997


## 1.2查看表格数据描述

In [4]:
df.describe()

,楼层,面积(㎡),价格(万元),年份
count,23677.000000,23677.000000,23677.000000,23677.000000
mean,12.764877,99.388616,610.668319,2001.326519
std,7.644138,51.270606,411.452107,9.001996
min,1.000000,15.000000,60.000000,1950.000000
25%,6.000000,66.000000,365.000000,1997.000000
50%,11.000000,88.000000,499.000000,2003.000000
75%,18.000000,118.000000,717.000000,2007.000000
max,57.000000,1019.000000,6000.000000,2017.000000


## 1.3查看表格是否有数据缺失

In [5]:
df.isnull().sum()

市区           0
小区           0
户型           0
朝向           0
楼层           0
装修情况         0
电梯        8257
面积(㎡)        0
价格(万元)       0
年份           0
dtype: int64

## 查看电梯列共有几种值

In [6]:
df.电梯.unique()

array(['有电梯', '无电梯', nan], dtype=object)

## 1.5缺失值补充

In [8]:
df.电梯.fillna('未知',inplace=True)
df.isnull().sum()

市区        0
小区        0
户型        0
朝向        0
楼层        0
装修情况      0
电梯        0
面积(㎡)     0
价格(万元)    0
年份        0
dtype: int64

## 1.6查看房屋朝向数据

In [9]:
df.朝向.unique()

array(['西', '南北', '东西', '南西', '西南', '东南', '南', '东北', '东', '西北', '北'],
      dtype=object)

In [10]:
df['朝向']=df.朝向.str.replace('南西','西南')
df.朝向.unique()

array(['西', '南北', '东西', '西南', '东南', '南', '东北', '东', '西北', '北'],
      dtype=object)

In [11]:
df_region=df.groupby('市区').agg(数量=('小区','count'))

## 1.7统计各城区二手房数量

In [12]:
df_region

,数量
市区,
东城,1533
丰台,2952
大兴,2115
密云,12
平谷,41
延庆,469
怀柔,15
房山,1442
昌平,2811


# 2 Pyecharts可视化

## 2.1北京各城区二手房数量地图分布

In [18]:
m=(Map()
  .add('北京市',[list(z) for z in zip([str(x)+'区' for x in df_region.index],df_region.数量)],'北京')
  .set_global_opts(title_opts=opts.TitleOpts(title='北京市各区二手房数量分布'),
                  visualmap_opts=opts.VisualMapOpts(max_=2973)))

m.render_notebook()



## 2.2各城区二手房数量-平均价格柱状图

In [19]:
df_price=df.groupby('市区').agg(平均价格=('价格(万元)','mean'))
df_price

,平均价格
市区,
东城,851.425245
丰台,525.103591
大兴,460.469693
密云,425.333333
平谷,308.658537
延庆,549.876333
怀柔,785.200000
房山,360.611859
昌平,469.230345


In [71]:
p2=(Bar()
   .add_xaxis(df_price.index.tolist())
   .add_yaxis('数量',df_region.数量.tolist())
   .extend_axis(yaxis=opts.AxisOpts(name='价格(万元)',axislabel_opts=opts.LabelOpts(formatter='{value}万元')))
   .set_global_opts(title_opts=opts.TitleOpts(title='各城区二手房数量-平均价格走势图'),
                   yaxis_opts=opts.AxisOpts(name='数量'),
                    xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_='shadow')),
                   toolbox_opts=opts.ToolboxOpts(is_show=True),
                   tooltip_opts=opts.TooltipOpts(is_show=True,trigger='axis',axis_pointer_type='cross'))
   .set_series_opts(label_opts=opts.LabelOpts(is_show=False)))

line=(Line()
     .add_xaxis(df_price.index.tolist())
     .add_yaxis('价格',df_price.平均价格.tolist(),yaxis_index=1,label_opts=opts.LabelOpts(is_show=False),z=10))
p2.overlap(line)

p2.render_notebook()

## 2.3二手房价格最高Top15

In [33]:
top_price=df.sort_values(by='价格(万元)',ascending=False)[:15]
top_price

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份
20390,西城,朱雀门,4室2厅,东南,5,其他,有电梯,376.0,6000.0,2008
22228,东城,贡院六号,5室2厅,南北,23,精装,有电梯,459.0,5500.0,2002
22907,东城,NAGA上院,6室2厅,东南,12,精装,有电梯,608.0,5000.0,2008
3219,顺义,丽宫,5室2厅,南北,3,精装,未知,685.0,5000.0,2007
22982,东城,当代MOMA,5室2厅,东南,7,精装,未知,384.0,4988.0,2006
20202,西城,耕天下,5室3厅,南北,7,其他,有电梯,330.0,4650.0,2003
6191,昌平,碧水庄园,5室3厅,南北,2,精装,未知,571.0,4600.0,2005
2391,顺义,丽嘉花园,4室2厅,东南,2,其他,未知,548.0,4500.0,2007
17285,朝阳,首府官邸,叠拼别墅,南北,5,精装,未知,523.4,4500.0,2007
15327,海淀,紫御府,4室2厅,南北,12,精装,有电梯,374.0,4368.0,2008


In [41]:
p3=(Bar(init_opts=opts.InitOpts(theme='essos'))
   .add_xaxis(top_price.小区.tolist())
   .add_yaxis('',top_price['价格(万元)'].values.tolist())
   .set_global_opts(yaxis_opts=opts.AxisOpts(name='价格(万元)'),
                   xaxis_opts=opts.AxisOpts(name='小区'),
                   title_opts=opts.TitleOpts(title='价格最高Top15所在小区'),
                   datazoom_opts=opts.DataZoomOpts(is_show=True)))
p3.render_notebook()

## 2.4二手房总价与面积散点图

In [43]:
p4=(Scatter()
   .add_xaxis(df['面积(㎡)'].values.tolist())
   .add_yaxis('',df['价格(万元)'].values.tolist())
   .set_global_opts(xaxis_opts=opts.AxisOpts(type_='value')))
p4.render_notebook()

## 2.5房屋朝向饼图

In [44]:
df_direction=df.groupby('朝向').agg(数量=('小区','count'))
df_direction

,数量
朝向,
东,934
东北,681
东南,2096
东西,1559
北,612
南,2847
南北,11433
西,889
西北,781


In [46]:
p5=(Pie()
   .add('',[list(z) for z in zip(df_direction.index.tolist(),df_direction.values.tolist())],radius=['20%','60%'])
   .set_global_opts(title_opts=opts.TitleOpts(title='房屋朝向占比'),
                   legend_opts=opts.LegendOpts(pos_left='2%',pos_top='20%',orient='vertical'))
   .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}:{c} ({d}%)')))
p5.render_notebook()

## 2.6装修情况/有无电梯玫瑰图


In [63]:
df_lift=df.groupby('电梯').agg(count=('小区','count')).reset_index()
df_fitment=df.groupby('装修情况').agg(count=('小区','count')).reset_index()
df_fitment




,装修情况,count
0,其他,3239
1,毛坯,583
2,简装,8499
3,精装,11356


In [70]:
p6=(Bar()
   .add_xaxis(df_fitment.装修情况.tolist())
   .add_yaxis('',df_fitment['count'].tolist(),category_gap='50%')
   .reversal_axis()
   .set_series_opts(label_opts=opts.LabelOpts(position='right'))
   .set_global_opts(xaxis_opts=opts.AxisOpts(name='数量'),
                   title_opts=opts.TitleOpts(title='装修情况/有无电梯玫瑰图'),
                    legend_opts=opts.LegendOpts(pos_left='90%',pos_top='50%'),
                   ))
c2=(Pie()
   .add('',[list(z)for z in zip(df_lift.电梯,df_lift['count'])],
       radius=['10%','30%'],
        center=['75%','60%'],
       rosetype='radius',
       label_opts=opts.LabelOpts(is_show=True))
   .set_global_opts(legend_opts=opts.LegendOpts(pos_left='90%',pos_top='50%'))
   .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}:{c} ({d}%)')))
p6.overlap(c2)
p6.render_notebook()